In [3]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

In [3]:
def relu_bn(inputs: tf.Tensor) -> tf.Tensor:
    x = layers.ReLU()(inputs)
    x = layers.BatchNormalization()(x)
    return x

def res_block(x: tf.Tensor, downsample:bool, filters:int, kernel_size = 3) -> tf.Tensor:
    y = layers.Conv2D(kernel_size=kernel_size, 
                               strides= 2 if downsample else 1, 
                               filters=filters, 
                               padding='same')(x)
    y = relu_bn(y)
    y = layers.Conv2D(kernel_size=kernel_size, 
                               strides= 1, 
                               filters=filters, 
                               padding='same')(y)
    if downsample:
        x = layers.Conv2D(kernel_size=1, 
        strides=2, 
        filters=filters)(x)
    y = layers.Add()([y, x])
    y = relu_bn(y)
    return y
        


In [4]:
def build_resnet(input_shape = (32,32,3), block_list = [2, 5, 5, 2], num_filters = 64, num_classes = 10):
    inputs = layers.Input(shape=input_shape)
    

    x = layers.BatchNormalization()(inputs)
    x = layers.Conv2D(kernel_size=3,
                        strides=1,
                        filters=num_filters,
                        padding='same')(x)
    x = relu_bn(x)

    for i in range(len(block_list)):
        num_blocks = block_list[i]
        for j in range(num_blocks):
            x = res_block(x, downsample=(j ==0 and i != 0), filters=num_filters)
        num_filters *= 2
    
    x = layers.AveragePooling2D()(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense( num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


In [5]:
"""
def ResNet18(input_shape = (32,32,3), num_classes = 10):
    return build_resnet(input_shape, [2,2,2,2], num_classes, num_filters=64)

def ResNet34(input_shape = (32,32,3), num_classes = 10):
    return build_resnet(input_shape, [3,4,6,3], num_classes, num_filters=64)

def ResNet50(input_shape = (32,32,3), num_classes = 10):
    return build_resnet(input_shape, [3,4,6,3], num_classes, num_filters=128) """

'\ndef ResNet18(input_shape = (32,32,3), num_classes = 10):\n    return build_resnet(input_shape, [2,2,2,2], num_classes, num_filters=64)\n\ndef ResNet34(input_shape = (32,32,3), num_classes = 10):\n    return build_resnet(input_shape, [3,4,6,3], num_classes, num_filters=64)\n\ndef ResNet50(input_shape = (32,32,3), num_classes = 10):\n    return build_resnet(input_shape, [3,4,6,3], num_classes, num_filters=128) '

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

model = build_resnet()
model.summary()

timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'cifar-10_res_net_30-'+timestr

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1
)
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/'+name,
    histogram_freq=1
)

model.fit(
    x=x_train,
    y=y_train,
    epochs=20,
    verbose=1,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[cp_callback, tensorboard_callback]
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 3)   12          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   1792        ['batch_normalization[0][0]']    
                                                                                                  
 re_lu (ReLU)                   (None, 32, 32, 64)   0           ['conv2d[0][0]']             

 rmalization)                                                                                     
                                                                                                  
 conv2d_10 (Conv2D)             (None, 16, 16, 128)  147584      ['batch_normalization_9[0][0]']  
                                                                                                  
 re_lu_9 (ReLU)                 (None, 16, 16, 128)  0           ['conv2d_10[0][0]']              
                                                                                                  
 batch_normalization_10 (BatchN  (None, 16, 16, 128)  512        ['re_lu_9[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv2d_11 (Conv2D)             (None, 16, 16, 128)  147584      ['batch_normalization_10[0][0]'] 
          

                                                                                                  
 re_lu_18 (ReLU)                (None, 8, 8, 256)    0           ['add_8[0][0]']                  
                                                                                                  
 batch_normalization_19 (BatchN  (None, 8, 8, 256)   1024        ['re_lu_18[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 conv2d_21 (Conv2D)             (None, 8, 8, 256)    590080      ['batch_normalization_19[0][0]'] 
                                                                                                  
 re_lu_19 (ReLU)                (None, 8, 8, 256)    0           ['conv2d_21[0][0]']              
                                                                                                  
 batch_nor

 conv2d_31 (Conv2D)             (None, 4, 4, 512)    2359808     ['batch_normalization_28[0][0]'] 
                                                                                                  
 add_13 (Add)                   (None, 4, 4, 512)    0           ['conv2d_31[0][0]',              
                                                                  'batch_normalization_27[0][0]'] 
                                                                                                  
 re_lu_28 (ReLU)                (None, 4, 4, 512)    0           ['add_13[0][0]']                 
                                                                                                  
 batch_normalization_29 (BatchN  (None, 4, 4, 512)   2048        ['re_lu_28[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 average_p

Ref:
- https://arxiv.org/pdf/1512.03385.pdf
- https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba


In [4]:
%debug magic

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

ipdb> c
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
NameError: name 'magic' is not defined
